**Pegar dados da planilha e organizar**

In [1]:
import pandas as pd
import re


def extract_and_clean_year(row):
    # Tentamos encontrar o padrão (XX/XX) ou XX/XX
    match = re.search(r'(\((\d{2}/\d{2})\)|(\d{2}/\d{2}))', row['Modelo'])
    if match:
        # Verifica qual dos padrões foi encontrado e pega o valor correto
        year = match.group(2) if match.group(2) else match.group(3)

        # Limpa a string da coluna 'Modelo' removendo o padrão de ano
        row['Modelo'] = row['Modelo'].replace(match.group(0), '').strip()
        row['Ano'] = year
    else:
        row['Ano'] = '23/23'
    return row


# Carregar o arquivo Excel
excel_file = pd.ExcelFile('D:/tabelaFiat2.xlsx')
all_sheet_names = excel_file.sheet_names

# Processar cada planilha e extrair os dados
all_sheets_data = []
for current_sheet_name in all_sheet_names[1:]:
    current_sheet_data = pd.read_excel(excel_file, sheet_name=current_sheet_name, header=None)
    sheet_rows_data = []

    current_title = None  # Aqui guardaremos o título/modelo atual
    for _, row in current_sheet_data.iterrows():
        # Verifica se encontramos um novo título
        if pd.notna(row[0]) and pd.isna(row[1]) and pd.isna(row[2]) and pd.isna(row[3]):
            current_title = row[0]
        # Caso contrário, verifica se as colunas 1, 2 e 3 têm dados (não são NA)
        elif pd.notna(row[1]) and pd.notna(row[2]) and pd.notna(row[3]):
            if row[1] == 'Tempo Padrão de Mão-De-Obra [horas]':                                 
                row_data = pd.DataFrame([[
                    row[1], row[2], row[3], current_title, 
                    row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13]
                ]], columns=[
                    'Peça', 'Código', 'Preço', 'Modelo', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'
                ])
            else:
                row_data = pd.DataFrame([[row[1], row[2], row[3], current_title]], columns=['Peça', 'Código', 'Preço', 'Modelo'])
            
            sheet_rows_data.append(row_data)

    all_sheets_data.append(pd.concat(sheet_rows_data))

                    
                    

# Concatenar todos os dataframes
df_excel = pd.concat(all_sheets_data)
df_excel = df_excel[df_excel['Peça'] == 'Tempo Padrão de Mão-De-Obra [horas]']
0
# Limpar e formatar o dataframe
df_excel['Preço'] = pd.to_numeric(df_excel['Preço'], errors='coerce')
df_excel.dropna(subset=['Preço'], inplace=True)
df_excel['Preço'] = df_excel['Preço'].round(2)
df_excel['Modelo'] = df_excel['Modelo'].str.replace('PALIO ', '', regex=False)
df_excel['Código'] = df_excel['Código'].astype(str)



# Aplicamos a função ao dataframe
df_excel = df_excel.apply(extract_and_clean_year, axis=1)

# Dividindo a coluna 'Ano' e criando 'ANO_INI' e 'ANO_FIM'.
df_excel[['ANO_INI', 'ANO_FIM']] = df_excel['Ano'].str.split('/', expand=True)

# Adicionando "20" na frente dos valores de 'ANO_INI' e 'ANO_FIM'
df_excel['ANO_INI'] = '20' + df_excel['ANO_INI']
df_excel['ANO_FIM'] = '20' + df_excel['ANO_FIM']


# Removendo a coluna 'Ano' original.
df_excel.drop('Ano', axis=1, inplace=True)

df_excel



,Peça,Código,Preço,Modelo,1,2,3,4,5,6,7,8,9,10,ANO_INI,ANO_FIM
0,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,CRONOS 1.0,0.85,1.20,0.90,2.55,1.00,1.45,0.85,2.55,0.90,1.35,2022,2023
0,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,CRONOS 1.3,0.85,1.20,0.90,2.55,1.00,1.45,0.85,2.55,0.90,1.35,2022,2022
0,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,CRONOS 1.3,0.85,1.20,0.90,2.55,1.00,1.45,0.85,2.55,0.90,1.35,2021,2022
0,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,CRONOS 1.3,0.85,1.20,0.90,2.55,1.00,1.45,0.85,2.55,0.90,1.35,2020,2021
0,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,CRONOS 1.8,0.85,1.20,0.90,2.05,1.00,2.55,0.90,2.05,0.90,1.35,2022,2022
0,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,CRONOS 1.8,0.85,1.20,0.90,2.05,1.00,2.55,0.90,2.05,0.90,1.35,2021,2022
0,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,CRONOS 1.8,0.85,1.20,0.90,2.05,1.00,2.55,0.90,2.05,0.90,1.35,2020,2021
0,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,UNO 1.0 FIRE,0.95,1.20,2.10,1.90,1.15,4.40,0.95,1.90,2.10,1.30,2021,2021
0,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,UNO 1.0 FIRE,0.95,1.20,2.10,1.90,1.15,4.40,0.95,1.90,2.10,1.30,2020,2021
0,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,UNO 1.0 FIREFLY,0.85,1.10,0.95,1.80,1.00,1.80,0.85,1.80,0.95,1.25,2021,2021


**Fazer o MELT de dados com as colunas dos tempos das revisoes**

In [2]:
df_melted = pd.melt(df_excel, id_vars=['Peça', 'Código', 'Preço', 'Modelo', 'ANO_INI', 'ANO_FIM'],
                    value_vars=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
                    var_name='Desc_Revisao', value_name='Tempo')

# Aqui estamos modificando os valores da coluna 'Desc_Revisao' para incluir " KM" no final
df_melted['Desc_Revisao'] = df_melted['Desc_Revisao'].apply(lambda x: f'REVISAO {x}0.000 KM')

#Juntar as colunas desc e modelo
df_melted['Descricao_revisao'] = df_melted['Desc_Revisao'] + ' ' + df_melted['Modelo']
#remover a coluna 'Modelo' do DataFrame df_melted
df_melted = df_melted.drop('Modelo', axis=1)
# remover a coluna 'Desc_Revisao' do DataFrame df_melted
df_melted = df_melted.drop('Desc_Revisao', axis=1)
# Exibindo as primeiras linhas do novo DataFrame como exemplo
df_melted


,Peça,Código,Preço,ANO_INI,ANO_FIM,Tempo,Descricao_revisao
0,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,2022,2023,0.85,REVISAO 10.000 KM CRONOS 1.0
1,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,2022,2022,0.85,REVISAO 10.000 KM CRONOS 1.3
2,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,2021,2022,0.85,REVISAO 10.000 KM CRONOS 1.3
3,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,2020,2021,0.85,REVISAO 10.000 KM CRONOS 1.3
4,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,2022,2022,0.85,REVISAO 10.000 KM CRONOS 1.8
...,...,...,...,...,...,...,...
535,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,2021,2022,1.90,REVISAO 100.000 KM TORO 2.0
536,Tempo Padrão de Mão-De-Obra [horas],MO FIAT,299,2023,2023,1.90,REVISAO 100.000 KM TORO 2.0
537,Tempo Padrão de Mão-De-Obra [horas],MO X250,229,2023,2023,2.70,REVISAO 100.000 KM DUCATO X250 MY´23
538,Tempo Padrão de Mão-De-Obra [horas],MO VAN,210,2023,2023,3.50,REVISAO 100.000 KM DUCATO


**CONEXAO COM BANCO**

In [3]:
import pyodbc
import pandas as pd

# Substitua os seguintes valores pelas suas credenciais de banco de dados
server = 'RAFAELPC' 
database = 'dbfacamp' 
driver = '{ODBC Driver 17 for SQL Server}'

# String de conexão
conn_string = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Conecte ao banco de dados
conn = pyodbc.connect(conn_string)

# Crie um cursor a partir da conexão
cursor = conn.cursor()

Query com os dados que vou atualizar


In [11]:
# Criar tabela temporária
cursor.execute('''
IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'temp_table')
BEGIN
    DROP TABLE temp_table;
END
CREATE TABLE temp_table (
    Peca VARCHAR(255),
    Codigo VARCHAR(255),
    Preco FLOAT,
    ANO_INI INT,
    ANO_FIM INT,
    Tempo FLOAT,
    Descricao_revisao VARCHAR(255)
);

''')
conn.commit()


    

In [12]:
# Inserir dados do DataFrame na tabela temporária
for index, row in df_melted.iterrows():
    cursor.execute('''
        INSERT INTO temp_table (Peca, Codigo, Preco, ANO_INI, ANO_FIM, Tempo, Descricao_revisao)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', row['Peça'], row['Código'], row['Preço'], row['ANO_INI'], row['ANO_FIM'], row['Tempo'], row['Descricao_revisao'])
    conn.commit()

In [14]:

cursor.execute('''
update t1
set t1.VAL_HORA = temp.Preco,
	t1.TEMPO = temp.Tempo

FROM KITS_SERV t1

JOIN KITS t2
ON t1.ID_KIT = t2.ID_KIT

INNER JOIN temp_table temp 
ON t2.PADRAO_DESCRICAO LIKE temp.Descricao_Revisao + '%'

where
t2.PADRAO_DESCRICAO  LIKE temp.Descricao_Revisao + '%'
and t2.ANO_INI <= temp.ANO_INI
AND t2.ANO_FIM >= temp.ANO_FIM;

''')
conn.commit()

# Fechar cursor e conexão
cursor.close()
conn.close()

